In [3]:
import pandas as pd
from Bio import SeqIO

Маленькая функция чтобы не копировать код

In [4]:
def save_bed(df: pd.DataFrame, name: str):
    df.to_csv(f'{name}.bed', header=False, index=False, sep='\t')

# ZDNABERT

Разделим наш геном на три части, и сделаем три запуска ZDNABERT

In [1]:
%%bash
seqkit split2 -p 3 data/remanei_genomic.fa -O data/splitted_genome

[INFO] split seqs from data/remanei_genomic.fa
[INFO] split into 3 parts
[INFO] write 1224 sequences to file: data/splitted_genome/remanei_genomic.part_001.fa
[INFO] write 1223 sequences to file: data/splitted_genome/remanei_genomic.part_002.fa
[INFO] write 1223 sequences to file: data/splitted_genome/remanei_genomic.part_003.fa


ZDNABERT запускался в колабе в отдельном ноутбуке от авторов статьи: https://colab.research.google.com/github/mitiau/Z-DNABERT/blob/main/ZDNA-prediction.ipynb

Сюда перенесены просто его выходы:

In [47]:
%%bash

ls zdnabert_results

text_predictions_pt1.txt
text_predictions_pt2.txt
text_predictions_pt3.txt


Распарсим его выходы и сохраним в bed файл

In [9]:
def parse_zdnabert_output_file(filepath):
    result = []
    with open(filepath, 'r') as file:
        file.readline()
        contig = None
        for line in file:
            parts = line.split()
            if parts[0].startswith('Crem_Contig'):
                contig = parts[0]
            elif len(parts) == 2 and parts[0].isdigit() and parts[1].isdigit():
                result.append({
                    'Contig': contig,
                    'start': int(parts[0]),
                    'end': int(parts[1]),
                })
            elif parts[0] != 'start':
                print(f'Undefined line: {line}')
                
    return result

all_files_result = []
for i in range(1, 4):
    filepath = f'zdnabert_results/text_predictions_pt{i}.txt'
    all_files_result.extend(parse_zdnabert_output_file(filepath))
    
zdnabert_df = pd.DataFrame(all_files_result)
save_bed(zdnabert_df, 'zdnabert_result')
zdnabert_df.sample(3)

,Contig,start,end
1860,Crem_Contig238,18428,18444
376,Crem_Contig0,4296352,4296363
1480,Crem_Contig105,208358,208371


# Z-HUNT

Соберем Z-Hunt


**!!!Важно!!!** отметить, что тут используется версия программы куда уже вшито отсичения результатов по скору, потому она работает быстрее и ничего дополнительно фильтровать не нужно

In [4]:
%%bash
gcc zhunt3-alan.c -lm -o zhunt3

zhunt3-alan.c: In function ‘user_regret’:
zhunt3-alan.c:303:7: warning: implicit declaration of function ‘gets’; did you mean ‘fgets’? [-Wimplicit-function-declaration]
  303 |       gets( tempstr );
      |       ^~~~
      |       fgets
/usr/bin/ld: /tmp/ccaddCZW.o: in function `user_regret':
zhunt3-alan.c:(.text+0xbd3): warning: the `gets' function is dangerous and should not be used.


Теперь разделим геном уже на 6 частей

In [2]:
%%bash
seqkit split2 -p 6 data/remanei_genomic.fa -O data/splitted_genome_6

[INFO] split seqs from data/remanei_genomic.fa
[INFO] split into 6 parts
[INFO] write 612 sequences to file: data/splitted_genome_6/remanei_genomic.part_001.fa
[INFO] write 612 sequences to file: data/splitted_genome_6/remanei_genomic.part_002.fa
[INFO] write 612 sequences to file: data/splitted_genome_6/remanei_genomic.part_003.fa
[INFO] write 612 sequences to file: data/splitted_genome_6/remanei_genomic.part_004.fa
[INFO] write 611 sequences to file: data/splitted_genome_6/remanei_genomic.part_005.fa
[INFO] write 611 sequences to file: data/splitted_genome_6/remanei_genomic.part_006.fa


Запустим параллельную обработку с помощью Z-Hunt

In [1]:
%%bash
parallel -j 6 './zhunt3 12 8 12 {}' ::: data/splitted_genome_6/remanei_genomic.part_{001..006}.fa

dinucleotides 12
min/max 8 12
min/max 8 12
operating on data/splitted_genome_6/remanei_genomic.part_001.fa
calculating zscore
opening data/splitted_genome_6/remanei_genomic.part_001.fa
inputting sequence
opening data/splitted_genome_6/remanei_genomic.part_001.fa.Z-SCORE

 run time=2272 sec
use min/max 8 12
analyzing_zscore
opening data/splitted_genome_6/remanei_genomic.part_001.fa.Z-SCORE
opening data/splitted_genome_6/remanei_genomic.part_001.fa
inputting sequence
dinucleotides 12
min/max 8 12
min/max 8 12
operating on data/splitted_genome_6/remanei_genomic.part_005.fa
calculating zscore
opening data/splitted_genome_6/remanei_genomic.part_005.fa
inputting sequence
opening data/splitted_genome_6/remanei_genomic.part_005.fa.Z-SCORE

 run time=2983 sec
use min/max 8 12
analyzing_zscore
opening data/splitted_genome_6/remanei_genomic.part_005.fa.Z-SCORE
opening data/splitted_genome_6/remanei_genomic.part_005.fa
inputting sequence
dinucleotides 12
min/max 8 12
min/max 8 12
operating on data

Опять таки распарсим выходы Z-Hunt и обьединим выходы в один файл

In [ ]:
def make_contig_to_position_dict(genomic_file):
    contigs_bounds = []
    cumsum = 1
    for record in SeqIO.parse(genomic_file, "fasta"):
        contigs_bounds.append((record.id, cumsum, cumsum + len(str(record.seq)) - 1))
        cumsum += len(str(record.seq))
        
    result = []
    
    zhunt_output = f'{genomic_file}.Z-SCORE'
    
    with open(zhunt_output, 'r') as f:
        next(f)
        zhunt_pairs = [tuple(map(int, line.split()[:2])) for line in f]

    zhunt_pairs.sort(key=lambda x: x[0])
    
    current_contig_idx = 0
    unreached_lines = 0
    total_lines = 0
    picked_lines = 0
    for begin, end in zhunt_pairs:
        total_lines += 1            
        while current_contig_idx < len(contigs_bounds) and begin > contigs_bounds[current_contig_idx][2]:
            current_contig_idx += 1
            
        if current_contig_idx >= len(contigs_bounds):
            break
            
        contig, contig_begin, contig_end = contigs_bounds[current_contig_idx]
        
        if contig_begin <= begin and end <= contig_end:
            picked_lines += 1
            result.append({
                'Contig': contig,
                'start': begin - contig_begin,
                'end': end - contig_begin
            })
        
    print(f'    {picked_lines} lines added to bed file out of {total_lines}')
                
    return result

all_files_zhunt_result = []
for i in range(1, 7):
    filepath = f'data/splitted_genome_6/remanei_genomic.part_00{i}.fa'
    print(f'Parsing {filepath}')
    all_files_zhunt_result.extend(make_contig_to_position_dict(filepath))
    
zhunt_df = pd.DataFrame(all_files_zhunt_result)
save_bed(zhunt_df, 'zhunt_result')
zhunt_df

Parsing data/splitted_genome_6/remanei_genomic.part_001.fa
    22699 lines added to bed file out of 22724
Parsing data/splitted_genome_6/remanei_genomic.part_002.fa
    34725 lines added to bed file out of 34752
Parsing data/splitted_genome_6/remanei_genomic.part_003.fa
    35129 lines added to bed file out of 35134
Parsing data/splitted_genome_6/remanei_genomic.part_004.fa
    30970 lines added to bed file out of 30993
Parsing data/splitted_genome_6/remanei_genomic.part_005.fa
    26947 lines added to bed file out of 26972
Parsing data/splitted_genome_6/remanei_genomic.part_006.fa
    26334 lines added to bed file out of 26338


,Contig,start,end
0,Crem_Contig5428,798,816
1,Crem_Contig5428,800,816
2,Crem_Contig5428,802,818
3,Crem_Contig5428,804,820
4,Crem_Contig5428,947,971
...,...,...,...
176799,Crem_Contig1426,9952,9970
176800,Crem_Contig1426,9953,9969
176801,Crem_Contig1426,9954,9970
176802,Crem_Contig1426,9955,9971


Поскольку Z-Hunt выдает пересекающиеся предсказания сделаем merge

In [6]:
%%bash
bedtools sort -i zhunt_result.bed > zhunt_result.sorted.bed
bedtools merge -i zhunt_result.sorted.bed > zhunt_result.merged.bed

# Квадруплексы

Теперь ищем квадруплексы для обоих стрендов по известным regex. Результат снова сохраним в bed файл

In [11]:
import re

pattern_plus = r"((?:G{3,}[ATGC]{1,7}){3,}G{3,})"
pattern_minus = r"((?:C{3,}[ATGC]{1,7}){3,}C{3,})"

def compile_patterns(plus: str, minus: str):
    return [
        (re.compile(plus, re.IGNORECASE), '+'),
        (re.compile(minus, re.IGNORECASE), '-')
    ]

def find_pqs(input_path, patterns):
    rows = []
    for record in SeqIO.parse(input_path, 'fasta'):
        seq = str(record.seq)
        for regex, strand in patterns:
            for m in regex.finditer(seq):
                rows.append({
                    'Scaffold': record.id,
                    'Start': m.start(),
                    'End': m.end(),
                    'Name': '',
                    'Score': '',
                    'Strand': strand
                })
    return pd.DataFrame(rows, columns=['Scaffold', 'Start', 'End', 'Name', 'Score', 'Strand'])

patterns = compile_patterns(pattern_plus, pattern_minus)
quad_results = find_pqs('data/remanei_genomic.fa', patterns)
save_bed(quad_results, 'quad_result')
quad_results

,Scaffold,Start,End,Name,Score,Strand
0,Crem_Contig264,51703,51718,,,+
1,Crem_Contig264,77587,77603,,,+
2,Crem_Contig264,113783,113806,,,+
3,Crem_Contig264,13057,13079,,,-
4,Crem_Contig264,49351,49367,,,-
...,...,...,...,...,...,...
4724,Crem_Contig75,358660,358683,,,-
4725,Crem_Contig75,384841,384856,,,-
4726,Crem_Contig1067,987,1015,,,+
4727,Crem_Contig1067,695,711,,,-


# Составление таблиц

Все данные готовы, можно начинать анализировать, сначала прочитаем нашу аннотацию

In [2]:
gff_cols = ['seqid','source','type','start','end','score','strand','phase','attributes']
gff = pd.read_csv(
    'data/remanei_annotations.gff3',
    sep='\t',
    comment='#',
    header=None,
    names=gff_cols
)

Достанем из нее экзоны и гены

In [5]:
exons = gff[gff['type'] == 'exon'].copy()
exons['start0'] = exons['start'] - 1
exons_bed = exons[['seqid', 'start0', 'end']].sort_values(['seqid', 'start0'])
exons_bed.to_csv('exons.bed', sep='\t', header=False, index=False)
save_bed(exons_bed, 'exons')

genes = gff[gff['type'] == 'gene'].copy()
genes['start0'] = genes['start'] - 1
genes_bed = genes[['seqid', 'start0', 'end']] \
    .sort_values(['seqid', 'start0'])
genes_bed.to_csv('genes.bed', sep='\t', header=False, index=False)
save_bed(genes_bed, 'genes')

print("Exons BED:")
print(exons_bed.head().to_string(index=False))
print("\nGenes BED:")
print(genes_bed.head().to_string(index=False))

Exons BED:
       seqid  start0  end
Crem_Contig0     230  362
Crem_Contig0     411  546
Crem_Contig0     593  736
Crem_Contig0     787  873
Crem_Contig0     920 1012

Genes BED:
       seqid  start0   end
Crem_Contig0     230  1012
Crem_Contig0    3678  5874
Crem_Contig0    9033 10647
Crem_Contig0   13157 13948
Crem_Contig0   15296 15939


Интроны это гены без экзонов, значит достаточно просто сделать subtract

In [6]:
%%bash
sort -k1,1 -k2,2n genes.bed  > genes.sorted.bed
sort -k1,1 -k2,2n exons.bed  > exons.sorted.bed

bedtools subtract -a genes.sorted.bed -b exons.sorted.bed > introns.bed

head introns.bed

Crem_Contig0	362	411
Crem_Contig0	546	593
Crem_Contig0	736	787
Crem_Contig0	873	920
Crem_Contig0	3729	4102
Crem_Contig0	4178	4224
Crem_Contig0	4284	4329
Crem_Contig0	4398	4802
Crem_Contig0	5299	5736
Crem_Contig0	9162	9240


Теперь создадим удобный bed файл с колонкой имени и стрэнда, стрнэнд нам нужен чтобы определять направление, например для поиска downstream и промотеров, а имя просто для удобства выполнения групповой части

In [40]:
import re

def extract_name(attr):
    m = re.search(r"Name=([^;]+)", attr)
    return m.group(1) if m else "unknown"

genes6 = genes[['seqid','start0','end','strand']].copy()
genes6['name']  = genes['attributes'].apply(extract_name)
genes6['score'] = '0'
save_bed(genes6[['seqid','start0','end','name','score','strand']], 'genes6')

Посчитаем длины всех контигов в геноме

In [12]:
%%bash

samtools faidx data/remanei_genomic.fa -o remanei_genomic.fai
cut -f1,2 remanei_genomic.fai > contig.sizes

head -n 3 contig.sizes

Crem_Contig5428	2460
Crem_Contig1821	4048
Crem_Contig1114	11957


Промотеры берем как 1000 пар оснований справа от гена

In [41]:
%%bash

bedtools flank -i genes6.bed -g contig.sizes -l 1000 -r 0 -s > promoters.bed

head -n 3 promoters.bed

Crem_Contig0	1012	2012	WBGene00051210	0	-
Crem_Contig0	2678	3678	WBGene00051212	0	+
Crem_Contig0	8033	9033	WBGene00051213	0	+


Downstream участки как 200 пар основания справа от генов

In [36]:
%%bash

bedtools flank -i genes6.bed -g contig.sizes -l 0 -r 200 -s > downstream.bed

head -n 3 downstream.bed

Crem_Contig0	30	230	WBGene00051210	0	-
Crem_Contig0	5874	6074	WBGene00051212	0	+
Crem_Contig0	10647	10847	WBGene00051213	0	+


Межгенные участки найдем просто найдя дополнения до уже узвестных нам участков генов

In [37]:
%%bash

bedtools sort -i genes.bed -g contig.sizes > genes_sorted_as_contig.bed

bedtools complement -i genes_sorted_as_contig.bed -g contig.sizes > intergenic.bed

head -n 3 intergenic.bed

Crem_Contig5428	0	2460
Crem_Contig1821	0	1116
Crem_Contig1821	4047	4048


In [27]:
%%bash

sort -k1,1 -k2,2n quad_result.bed > quad_result.sorted.bed
sort -k1,1 -k2,2n zhunt_result.merged.bed > zhunt_result.merged.sorted.bed
sort -k1,1 -k2,2n zdnabert_result.bed > zdnabert_result.sorted.bed

In [12]:
%%bash

cp quad_result.sorted.bed quad.bed
cp zhunt_result.merged.sorted.bed zhunt.bed
cp zdnabert_result.sorted.bed zdnabert.bed

Теперь пересечем все интерисующие нас участки с выходами zhunt, ZDNABERT и найденными квадруплексами

In [34]:
def count_lines(path):
    with open(path, 'r', encoding='utf-8') as f:
        return sum(1 for _ in f)

results = {
  'Quadruplex': 'quad.bed',
  'Z-Hunt': 'zhunt.bed',
  'ZDNABERT': 'zdnabert.bed',
}

positions = [
  'genes',
  'exons',
  'introns',
  'promoters',
  'downstream',
  'intergenic',
]

table1_df = pd.DataFrame(columns=results.keys(), index=positions)
table2_df = pd.DataFrame(columns=results.keys(), index=positions)

for result_name, result_file in results.items():
    for position in positions:
      table1_intersection_result_file = f'tables/table1_{position}_{result_name}.bed'
      table2_intersection_result_file = f'tables/table2_{position}_{result_name}.bed'
      
      !bedtools intersect -a {result_file} -b {position}.bed -u > {table1_intersection_result_file}
      !bedtools intersect -a {position}.bed -b {result_file} -u > {table2_intersection_result_file}
      
      table1_df.loc[position, result_name] = count_lines(table1_intersection_result_file)
      table2_df.loc[position, result_name] = count_lines(table2_intersection_result_file)

In [35]:
for structure in results.keys():
    table1_df[f'{structure} count'] = table1_df[structure]
    table1_df[f'{structure} fraction'] = table1_df[structure] / table1_df[structure].sum()
    
for structure in results.keys():
    table2_df[f'Segments with {structure} prediction'] = table2_df[structure]

Посчитаем общую длину геном для составления фона для первой таблицы

In [38]:
%%bash

grep -v '^>' data/remanei_genomic.fa | tr -d '\n' | wc -c

145442736


Смерджим интервалы внутри участков, чтобы посчитать точную суммарную длину участков для каждого участка генома

In [43]:
for position in positions:
    !sort -k1,1 -k2,2n {position}.bed  > {position}.sorted.bed
    !bedtools merge -i {position}.sorted.bed > {position}.merged.bed

Посчитаем фон для нашей первой таблицы, чтобы сравнить

In [44]:
genome_length = 145442736

def sum_bed_lengths(bed_path):
    df = pd.read_csv(
        bed_path,
        sep='\t',
        header=None,
    )
    return int((df[2] - df[1]).sum())

table1_df['background'] = table1_df.apply(lambda row: sum_bed_lengths(f'{row.name}.merged.bed') / genome_length, axis=1)

Выведем полученные таблицы и сохраним результат

In [45]:
print(table1_df.drop(columns=[structure for structure in results.keys()]).to_markdown())

|            |   Quadruplex count |   Quadruplex fraction |   Z-Hunt count |   Z-Hunt fraction |   ZDNABERT count |   ZDNABERT fraction |   background |
|:-----------|-------------------:|----------------------:|---------------:|------------------:|-----------------:|--------------------:|-------------:|
| genes      |               1502 |             0.200561  |          10140 |         0.2613    |             4247 |           0.234188  |    0.478726  |
| exons      |                286 |             0.0381893 |           5869 |         0.151239  |             2459 |           0.135594  |    0.266548  |
| introns    |               1243 |             0.165977  |           5132 |         0.132248  |             1978 |           0.109071  |    0.21358   |
| promoters  |                997 |             0.133129  |           4398 |         0.113333  |             2584 |           0.142487  |    0.200613  |
| downstream |                226 |             0.0301776 |            967 |      

In [46]:
print(table2_df.drop(columns=[structure for structure in results.keys()]).to_markdown())

|            |   Segments with Quadruplex prediction |   Segments with Z-Hunt prediction |   Segments with ZDNABERT prediction |
|:-----------|--------------------------------------:|----------------------------------:|------------------------------------:|
| genes      |                                  1104 |                              6666 |                                2826 |
| exons      |                                   294 |                              5421 |                                2055 |
| introns    |                                   933 |                              4253 |                                1606 |
| promoters  |                                   980 |                              4036 |                                2501 |
| downstream |                                   232 |                               944 |                                 299 |
| intergenic |                                  2264 |                              6928 |       

In [48]:
table1_df.to_csv('tables/table1.csv')
table2_df.to_csv('tables/table2.csv')